In [1]:
include("Axis_Aligned_Box.jl");

In [2]:
type Mondrian_Node
    parent::Nullable{Mondrian_Node}
    left::Nullable{Mondrian_Node}
    right::Nullable{Mondrian_Node}
    τ::Float64
    node_type::Array{Bool,1}        # node,leaf,root
    δ::Nullable{Int}                # split dimension
    ζ::Nullable{Float64}            # split position
end

function Mondrian_Node()
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      0,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}())
    return N
end

function Mondrian_Node(τ::Float64)
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,[true,false,false]
                      Nullable{Int}(),
                      Nullable{Float64}())
    return N
end

function Mondrian_Node(τ::Float64, node_type::Array{Bool,1})
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,
                      node_type
                      Nullable{Int}(),
                      Nullable{Float64}())
    return N
end

LoadError: [91msyntax: missing comma or ) in argument list[39m

In [3]:
c = Mondrian_Node()

Mondrian_Node(Nullable{Mondrian_Node}(), Nullable{Mondrian_Node}(), Nullable{Mondrian_Node}(), 0.0, Bool[true, false, false], Nullable{Int64}(), Nullable{Float64}())

In [51]:
using Distributions

function Sample_Mondrian_Tree(λ,D)
    δ = []
    ζ = []
    n = 1:size(D,1)
    e = Mondrian_Node(0.0,[false,false,true])
    Θ = Axis_Aligned_Box(get_intervals(D[1]))
    Sample_Mondrian_Block(e, Θ, λ, δ, ζ)
    return e
end

function Sample_Mondrian_Block(j, Θ, λ, δ, ζ)
    E = rand(Exponential(1/Linear_dimension(Θ)))
    if j.node_type[3]==true
        τₚ = 0
    else
        τₚ = (get(j.parent)).τ
    end
    if τₚ + E < λ
        d,x = sample_split_dimension(Θ)
        #push!(δ,d)
        #push!(ζ,x)
        Θᴸ = copy(Θ)
        Θᴿ = copy(Θ)
        Θᴸ.Intervals[d,2]=x
        Θᴿ.Intervals[d,1]=x
        left = Mondrian_Node(τₚ+E, [true,false,false])
        left.parent = j
        right = Mondrian_Node(τₚ+E, [true,false,false])
        right.parent = j
        j.left = left
        j.right = right
        Sample_Mondrian_Block(left, Θᴸ, λ, δ, ζ)
        Sample_Mondrian_Block(right, Θᴿ,λ, δ, ζ)
    else
        j.τ = λ
        j.node_type = [false,true,false]
    end
end

Sample_Mondrian_Block (generic function with 2 methods)

In [52]:
X = rand(10,10)
Y = rand(10)
D = (X,Y)

([0.372728 0.516704 … 0.998043 0.233049; 0.185573 0.269935 … 0.465681 0.614883; … ; 0.96987 0.625297 … 0.732517 0.46793; 0.105781 0.791417 … 0.721722 0.581104], [0.311622, 0.49522, 0.030827, 0.793182, 0.193743, 0.884442, 0.359504, 0.738085, 0.915334, 0.125943])

In [53]:
M = Sample_Mondrian_Tree(1,D);